

# Intel® Distribution of OpenVINO™ toolkit hetero plugin


    
This example shows how to use hetero plugin to define preferences to run different network layers on different hardware types. Here, we will use the command line option to define hetero plugin usage where the layer distribution is already defined. However, hetero plugin also allows developers to customize distribution of layers execution on different hardware by specifying it in the application code.

## Car detection tutorial example

### 1. Importing dependencies, Setting the Environment variables and Generate the IR files

In [ ]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *

In [ ]:
!/opt/intel/openvino/bin/setupvars.sh

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd  -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1  -o models

In [ ]:
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/mobilenet-ssd/FP32/ --scale 256 --mean_values [127,127,127]
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/squeezenet1.1/squeezenet1.1.caffemodel -o models/classification/squeezenet/1.1/caffe/  --scale 256 --mean_values [127,127,127]

In [ ]:
!make

In [ ]:
!./tutorial1 -h



### 2. Run the car detection tutorial with hetero plugin


#### Create Job Script 

We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file will be executed directly on the edge compute node.

In [ ]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE
ROIFILE=$OUTPUT_FILE/ROIs.txt
OVIDEO=$OUTPUT_FILE/output.mp4

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs - Updated for OpenVINO 2020.1
    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/BSP/a10_1150_sg1/linux64/lib
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R4_PL1_FP11_MobileNet_Clamp.aocx
    export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
fi

if [ "$FP_MODEL" = "FP32" ]; then
    config_file="conf_fp32.txt"
else
    config_file="conf_fp16.txt"
fi

# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
echo $FP_MODEL
./tutorial1 -i cars_1900.mp4 \
            -m models/mobilenet-ssd/$FP_MODEL/mobilenet-ssd.xml \
            -d $DEVICE \
            -o $OUTPUT_FILE\
            -fr 3000 

# Converting the text output to a video
./ROI_writer -i cars_1900.mp4 \
             -o $OUTPUT_FILE \
             -ROIfile $ROIFILE \
             -l pascal_voc_classes.txt \
             -r 2.0 # output in half res

#### a) Prioritizing running on GPU first.

In [ ]:
os.environ["VIDEO"] = "cars_1900.mp4"

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/GPU HETERO:GPU,CPU FP32 $VIDEO 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/GPU', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/GPU', 'v_progress_'+job_id_gpu[0]+'.txt', "Rendering", 0, 100)
    
while True:
    var=job_id_gpu[0].split(".")
    file="obj_det_gpu.o"+var[0]
    if os.path.isfile(file): 
        ! cat $file
        break


    
#### b) Prioritizing running on CPU first.

In [ ]:
#Submit job to the queue
job_id_cpu = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/Core HETERO:CPU,GPU FP32 $VIDEO 4" -N obj_det_cpu 
print(job_id_cpu[0]) 
if job_id_cpu:
     progressIndicator('results/Core', 'i_progress_'+job_id_cpu[0]+'.txt', "Inference", 0, 100)
     progressIndicator('results/Core', 'v_progress_'+job_id_cpu[0]+'.txt', "Rendering", 0, 100)
    
while True:
    var=job_id_cpu[0].split(".")
    file="obj_det_cpu.o"+var[0]
    if os.path.isfile(file): 
        ! cat $file
        break


Observe the performance time required to process each frame by Inference Engine. For this particular example, inference ran faster when prioritized for CPU as oppose to when GPU was the first priority.